# Verifying the Integrity of the Kaspa Chain
### Authors: Shai Wyborski, Michael Sutton

The purpose of this notebook is to guide you through the process of verifying the authenticity of the Kaspa chain. That is, to show you how to cryptographically verify that the current UTXO set of Kaspa has naturally evolved from an empty UTXO set, validating the assertion that there was no premine.

As you might know, about two weeks after mainnet launched the network a major bug caused the network to split. The effects of the bug started to appear during Nov' 22, 2021, and by Nov' 23, 2021 the network was completely fragmented. To restart the network after fixing the bug, many users were asked to provide their node data prior to the crash. The devs examined the data to isolate the latest possible block known to most of the network, the chosen block is the block with hash

>0fca37ca667c2d550a6c4416dad9717e50927128c424fa4edbebc436ab13aeef

whose timestamp is

>Monday, November 22, 2021 7:34:31.

We call this block the `checkpoint block`.

The new node was hardwired with a new genesis block whose UTXO set is identical to that of the block described above, which we call the `hardwired genesis`. The node data can only be used to authenticate the chain down to the hardwired genesis. (You can read more about the malfunction and how it was consolidated __[here](https://t.co/DhyAnxoU1t)__, __[here](https://t.co/MlxFT1svkX)__, and __[here](https://hashdag.medium.com/kaspa-black-tuesday-8c7f4fa35834)__. Also, it is impossible to prove from the node data alone that the UTXO set of the hardwired genesis indeed matches the UTXO set of the checkpoint block.

To fill in this gap, we asked the community to store copies of the data directory at the time of the crash (available e.g. __[here](https://mega.nz/file/rOJmhLIR#5j7wko32Mh0MlsQnC9yVG6jCvPql7Isqcyvgh3kmxKk)__). With this data available, all of the above could be verified.

The authentication process is as follows:
<ol>
<li>Verify the hashes of the pruning block chain. That is, verify that the hash of each header matches the hash stored in the header pointing at it (such checks seem tautological, e.g. it is "obvious" that "the block whose hash is x has hash x". However, relying on the assumption that the hash used to index the headers is the same as the hash used in consensus is insecure, and actually highly exploitable).</li>
<li>Reconstruct the coinbase transaction with the payload hardwired into the code and verify that it hashes correctly.</li>
<li>Verify that the hash of the checkpoint block matches the one advertised in Discord.</li>
<li>To provide additional proof that the hardwired genesis block was created at the time specified above, the hash of the most recent Bitcoin block was included in the data committed to the genesis block. This is a cryptographic version of taking a picture with today's newspaper. Look up the hash in Bitcoin's block explorer and verify that its timestamp matches the time the network resumed operation.</li>
<li>Recover the UTXO set of the checkpoint block from the stored data directory, compute its hash, and verify that it matches the UTXO commitment stored in the hardwired genesis header.</li>
<li>Recover the pruning headers from the snapshot, and repeat the first step going from the hardwired genesis block to the original genesis block.</li>
<li>Verify that the UTXO commitment in the original genesis block corresponds to an empty UTXOset.</li>
</ol>

We will now walk through the code that does all that (the code was written and run on a linux machine, running it on other operating syst.ms might require adaptations).

The first part of the code is not particularly interesting, just note that all of the imported directories are standard Python directories.

In [17]:
import os
import numpy as np
import pandas as pd
from store import *
import kbech32

The next code block implements the logic of hashing a header/transaction. It does nothing but serialize the contents of the structure and passing it to the standard implementation of blake. The dilligent reader might want to verify that the fields are serialized the same way in both implementations. However, this is not necessary, since if they are serialized differently and still produce the same hash this essentially means we found a collission in blake2b, which is believed to be collision resistant. 

In [2]:
import hashlib
import struct

# Relevant hashing functions as implemented by the golang 
# and rust kaspa codebases
# Note: uses only standart hash libs

def transaction_hash(t):
    hasher = hashlib.blake2b(digest_size=32, key=b"TransactionHash")
    hasher.update(struct.pack(f"<HQ", t.version, len(t.inputs)))
    for ti in t.inputs:
        hasher.update(ti.previousOutpoint.transactionID.transactionId)
        hasher.update(struct.pack(f"<IQ", ti.previousOutpoint.index, 
                                  len(ti.signatureScript)))
        hasher.update(ti.signatureScript)
        # Note: a subsequent HF added sig_op_count hashing here
        hasher.update(struct.pack(f"<Q", ti.sequence))

    hasher.update(struct.pack(f"<Q", len(t.outputs)))
    for to in t.outputs:
        hasher.update(struct.pack(f"<QHQ", to.value, 
                                  to.scriptPublicKey.version, 
                                  len(to.scriptPublicKey.script)))
        hasher.update(to.scriptPublicKey.script)
        
    hasher.update(struct.pack(f"<Q", t.lockTime))
    hasher.update(t.subnetworkID.subnetworkId)
    hasher.update(struct.pack(f"<QQ", t.gas, len(t.payload)))
    hasher.update(t.payload)
    return hasher.digest()

def header_hash(h):
    hasher = hashlib.blake2b(digest_size=32, key=b"BlockHash")
    hasher.update(struct.pack(f"<HQ", h.version, len(h.parents)))
    for level_parents in h.parents:
        hasher.update(struct.pack(f"<Q", len(level_parents.parentHashes)))
        for parent in level_parents.parentHashes:
            hasher.update(parent.hash)
    hasher.update(h.hashMerkleRoot.hash)    
    hasher.update(h.acceptedIDMerkleRoot.hash)
    hasher.update(h.utxoCommitment.hash)
    hasher.update(struct.pack(f"<QIQQQQ", 
                              h.timeInMilliseconds, 
                              h.bits, 
                              h.nonce, 
                              h.daaScore, 
                              h.blueScore, 
                              len(h.blueWork)))
    hasher.update(h.blueWork)
    hasher.update(h.pruningPoint.hash)
    return hasher.digest()

The next code block implements the logic to perform checks 1. and 7. above. Each pruning block stores the hash of the next pruning block, this code verifies that all pruning blocks indeed hash correctly to the hash stored in the next pruning block. Note that this check is not actually performed yet.

In [3]:
# Asserts a verified chaih of hashes from the givan block to the given genesis
def assert_cryptographic_hash_chain_to_genesis(
    store, 
    block_hash, 
    genesis_hash):
    
    i = 0
    while True:
        if block_hash == genesis_hash:
            print('Reached the queried genesis block: \n', 
                  genesis_hash.hex(), 'via', i, 'pruning points')
            return
        header = store.get_raw_header(block_hash)
        # Assert the block hash is correct
        assert(header_hash(header) == block_hash)
        block_hash = header.pruningPoint.hash
        i += 1

The next two blocks copy the data required from the golang implementation. By following the links in the comments one can verify that they match the data used by nodes after the hard-fork. At the very bottom of the second block we see that the checkpoint block hash indeed matches the hash posted to Discord. We can also see that the payload contains the hash of a __[Bitcoin block](https://blockstream.info/block/0000000000000000000b1f8e1c17b0133d439174e52efbb0c41c3583a8aa66b0)__ with the following timestamp:
> 2021-11-25 19:53:36 GMT +2

Which matches the time the network __[resumed operation](https://discord.com/channels/599153230659846165/844142778232864809/913445783011987508)__.
This completes checks 3. and 4. above, though note that this does not prove anything before check 2. is also complete.


In [4]:
# Take genesis hash from current go code
# Golang ref: 
# https://github.com/kaspanet/kaspad/blob/master/domain/dagconfig/genesis.go#L56

genesis_hash = bytes([
    0x58, 0xc2, 0xd4, 0x19, 0x9e, 0x21, 0xf9, 0x10, 
    0xd1, 0x57, 0x1d, 0x11, 0x49, 0x69, 0xce, 0xce, 
    0xf4, 0x8f, 0x9, 0xf9, 0x34, 0xd4, 0x2c, 0xcb, 
    0x6a, 0x28, 0x1a, 0x15, 0x86, 0x8f, 0x29, 0x99])


In [5]:
# Build genesis's coinbase tx payload, which references the pre-halt checkpoint 
# Golang ref: 
# https://github.com/kaspanet/kaspad/blob/master/domain/dagconfig/genesis.go#L18

genesis_tx_payload = bytes([
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, # Blue score
    0x00, 0xE1, 0xF5, 0x05, 0x00, 0x00, 0x00, 0x00, # Subsidy
    0x00, 0x00, # Script version
    0x01,                                           # Varint
    0x00,                                           # OP-FALSE
    
    # ומה די עליך ועל אחיך ייטב בשאר כספא ודהבה למעבד כרעות אלהכם תעבדון     
    0xd7, 0x95, 0xd7, 0x9e, 0xd7, 0x94, 0x20, 0xd7,
    0x93, 0xd7, 0x99, 0x20, 0xd7, 0xa2, 0xd7, 0x9c,
    0xd7, 0x99, 0xd7, 0x9a, 0x20, 0xd7, 0x95, 0xd7,
    0xa2, 0xd7, 0x9c, 0x20, 0xd7, 0x90, 0xd7, 0x97,
    0xd7, 0x99, 0xd7, 0x9a, 0x20, 0xd7, 0x99, 0xd7,
    0x99, 0xd7, 0x98, 0xd7, 0x91, 0x20, 0xd7, 0x91,
    0xd7, 0xa9, 0xd7, 0x90, 0xd7, 0xa8, 0x20, 0xd7,
    0x9b, 0xd7, 0xa1, 0xd7, 0xa4, 0xd7, 0x90, 0x20,
    0xd7, 0x95, 0xd7, 0x93, 0xd7, 0x94, 0xd7, 0x91,
    0xd7, 0x94, 0x20, 0xd7, 0x9c, 0xd7, 0x9e, 0xd7,
    0xa2, 0xd7, 0x91, 0xd7, 0x93, 0x20, 0xd7, 0x9b,
    0xd7, 0xa8, 0xd7, 0xa2, 0xd7, 0x95, 0xd7, 0xaa,
    0x20, 0xd7, 0x90, 0xd7, 0x9c, 0xd7, 0x94, 0xd7,
    0x9b, 0xd7, 0x9d, 0x20, 0xd7, 0xaa, 0xd7, 0xa2,
    0xd7, 0x91, 0xd7, 0x93, 0xd7, 0x95, 0xd7, 0x9f,
    
    # Bitcoin block hash 0000000000000000000b1f8e1c17b0133d439174e52efbb0c41c3583a8aa66b0
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x0b, 0x1f, 0x8e, 0x1c, 0x17, 0xb0, 0x13,
    0x3d, 0x43, 0x91, 0x74, 0xe5, 0x2e, 0xfb, 0xb0,
    0xc4, 0x1c, 0x35, 0x83, 0xa8, 0xaa, 0x66, 0xb0,
    
    # Checkpoint block hash 0fca37ca667c2d550a6c4416dad9717e50927128c424fa4edbebc436ab13aeef
    0x0f, 0xca, 0x37, 0xca, 0x66, 0x7c, 0x2d, 0x55,
    0x0a, 0x6c, 0x44, 0x16, 0xda, 0xd9, 0x71, 0x7e,
    0x50, 0x92, 0x71, 0x28, 0xc4, 0x24, 0xfa, 0x4e,
    0xdb, 0xeb, 0xc4, 0x36, 0xab, 0x13, 0xae, 0xef,
])


# Bitcoin explorer link: 
# https://blockstream.info/block/0000000000000000000b1f8e1c17b0133d439174e52efbb0c41c3583a8aa66b0
# 
# Kaspad version release: 
# https://github.com/kaspanet/kaspad/releases/tag/v0.11.5-2 
assert(bytes.fromhex('0000000000000000000b1f8e1c17b0133d439174e52efbb0c41c3583a8aa66b0') == 
       bytes([
           0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
           0x00, 0x0b, 0x1f, 0x8e, 0x1c, 0x17, 0xb0, 0x13,
           0x3d, 0x43, 0x91, 0x74, 0xe5, 0x2e, 0xfb, 0xb0,
           0xc4, 0x1c, 0x35, 0x83, 0xa8, 0xaa, 0x66, 0xb0,]))

assert(bytes.fromhex('0fca37ca667c2d550a6c4416dad9717e50927128c424fa4edbebc436ab13aeef') == 
       bytes([
           0x0f, 0xca, 0x37, 0xca, 0x66, 0x7c, 0x2d, 0x55,
           0x0a, 0x6c, 0x44, 0x16, 0xda, 0xd9, 0x71, 0x7e,
           0x50, 0x92, 0x71, 0x28, 0xc4, 0x24, 0xfa, 0x4e,
           0xdb, 0xeb, 0xc4, 0x36, 0xab, 0x13, 0xae, 0xef,]))


Next, we manually construct a coinbase transaction with the hash of the data above as a payload. Here, again, you could either verify that the computation is identical to the client code, or appeal to the fact that if it isn't, but the verification still passes, then we have identified a collision in blake2b.

In [6]:
# Build genesis's coinbase tx. 
# Golang ref: 
# https://github.com/kaspanet/kaspad/blob/master/domain/dagconfig/genesis.go#L51

genesis_coinbase_tx = type('Transaction', (object,), {})()
genesis_coinbase_tx.version = 0
genesis_coinbase_tx.subnetworkID = type('SubnetworkId', (object,), {})()
genesis_coinbase_tx.subnetworkID.subnetworkId = bytes.fromhex(
    '0100000000000000000000000000000000000000')
genesis_coinbase_tx.inputs = []
genesis_coinbase_tx.outputs = []
genesis_coinbase_tx.lockTime = 0
genesis_coinbase_tx.gas = 0
genesis_coinbase_tx.payload = genesis_tx_payload

The next block loads the ledger data from the synchronized node into the ```current_store``` variable, as well as the snapshot data into the ```pre_checkpoint_store``` variable.

In [8]:
pre_checkpoint_store = Store(r'/home/pool/data/kaspa-data-22-11-21-correct-utxo-commit')
current_store = Store(r'/home/pool/.kaspad/kaspa-mainnet/datadir2')

With all the required data and logic in place, we can start the verification process.

First, we obtain the genesis block and see that it hashes correctly.

In [18]:
genesis_header = current_store.get_raw_header(genesis_hash)

# Assert the genesis hash is correct
assert(header_hash(genesis_header) == genesis_hash)

We next verify that the hash of the coinbase transaction of the hardwired genesis block does match the hash of the coinbase transaction constructed above. Combined with the above, this completes checks 2-4.

In [10]:
# This shows that indeed current genesis refrences the checkpoint via the coinbase tx payload
assert(
    transaction_hash(genesis_coinbase_tx) == genesis_header.hashMerkleRoot.hash)

print(transaction_hash(genesis_coinbase_tx).hex()) 
print(genesis_header.hashMerkleRoot.hash.hex())

8ec898568c6801d13df4ee6e2a1b54b7e6236f671f20954f05306410518eeb32
8ec898568c6801d13df4ee6e2a1b54b7e6236f671f20954f05306410518eeb32


We next verify that the current chain of pruning points taken from the node leads to the hardwired genesis block. Completing check 1. above.

In [11]:
# Show that tips from current database link to genesis
tips, hst = current_store.tips()
assert_cryptographic_hash_chain_to_genesis(current_store, tips[0], genesis_hash)

Reached the queried genesis block: 
 58c2d4199e21f910d1571d114969cecef48f09f934d42ccb6a281a15868f2999 via 209 pruning points


We now see that the checkpoint block from the snapshot has the same UTXO state commitment as the genesis block, completing check 5.

In [12]:
# Now we move to the pre-halt database and show that the checkpoint was mined
# over the original genesis (with an empty UTXO-set commitment)

checkpoint_hash = bytes.fromhex(
    '0fca37ca667c2d550a6c4416dad9717e50927128c424fa4edbebc436ab13aeef')
checkpoint_header = pre_checkpoint_store.get_raw_header(checkpoint_hash)

# Assert the checkpoint hash is correct
assert(header_hash(checkpoint_header) == checkpoint_hash)

# Show that genesis and the checkpoint share the same UTXO commitment
assert(genesis_header.utxoCommitment.hash == checkpoint_header.utxoCommitment.hash)

print(genesis_header.utxoCommitment.hash.hex()) 
print(checkpoint_header.utxoCommitment.hash.hex())

710f27df423e63aa6cdb72b89ea5a06cffa399d66f167704455b5af59def8e20
710f27df423e63aa6cdb72b89ea5a06cffa399d66f167704455b5af59def8e20


Next, we repeat the check of the first step, only this time we are starting from the checkpoint block to arrive at the original genesis block, completing check 7.

In [120]:
# Golang ref from initial mainnet version: 
# https://github.com/kaspanet/kaspad/blob/v0.11.0/domain/dagconfig/genesis.go#L53C2-L56C49
original_genesis = bytes([
    0xca, 0xeb, 0x97, 0x96, 0x0a, 0x16, 0x0c, 0x21,
    0x1a, 0x6b, 0x21, 0x96, 0xbd, 0x78, 0x39, 0x9f,
    0xd4, 0xc4, 0xcc, 0x5b, 0x50, 0x9f, 0x55, 0xc1,
    0x2c, 0x8a, 0x7d, 0x81, 0x5f, 0x75, 0x36, 0xea,])

original_genesis_tx_payload = bytes([
	0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, # Blue score
	0x00, 0xE1, 0xF5, 0x05, 0x00, 0x00, 0x00, 0x00, # Subsidy
	0x00, 0x00, # Script version
	0x01,                                           # Varint
	0x00,                                           # OP-FALSE

    # ומה די עליך ועל אחיך ייטב בשאר כספא ודהבה למעבד כרעות אלהכם תעבדון
	0xd7, 0x95, 0xd7, 0x9e, 0xd7, 0x94, 0x20, 0xd7,
	0x93, 0xd7, 0x99, 0x20, 0xd7, 0xa2, 0xd7, 0x9c,
	0xd7, 0x99, 0xd7, 0x9a, 0x20, 0xd7, 0x95, 0xd7,
	0xa2, 0xd7, 0x9c, 0x20, 0xd7, 0x90, 0xd7, 0x97,
	0xd7, 0x99, 0xd7, 0x9a, 0x20, 0xd7, 0x99, 0xd7,
	0x99, 0xd7, 0x98, 0xd7, 0x91, 0x20, 0xd7, 0x91,
	0xd7, 0xa9, 0xd7, 0x90, 0xd7, 0xa8, 0x20, 0xd7,
	0x9b, 0xd7, 0xa1, 0xd7, 0xa4, 0xd7, 0x90, 0x20,
	0xd7, 0x95, 0xd7, 0x93, 0xd7, 0x94, 0xd7, 0x91,
	0xd7, 0x94, 0x20, 0xd7, 0x9c, 0xd7, 0x9e, 0xd7,
	0xa2, 0xd7, 0x91, 0xd7, 0x93, 0x20, 0xd7, 0x9b,
	0xd7, 0xa8, 0xd7, 0xa2, 0xd7, 0x95, 0xd7, 0xaa,
	0x20, 0xd7, 0x90, 0xd7, 0x9c, 0xd7, 0x94, 0xd7,
	0x9b, 0xd7, 0x9d, 0x20, 0xd7, 0xaa, 0xd7, 0xa2,
	0xd7, 0x91, 0xd7, 0x93, 0xd7, 0x95, 0xd7, 0x9f,

    # Bitcoin block hash 00000000000000000001733c62adb19f1b77fa0735d0e11f25af36fc9ca908a5
	0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
	0x00, 0x01, 0x73, 0x3c, 0x62, 0xad, 0xb1, 0x9f,
	0x1b, 0x77, 0xfa, 0x07, 0x35, 0xd0, 0xe1, 0x1f,
	0x25, 0xaf, 0x36, 0xfc, 0x9c, 0xa9, 0x08, 0xa5,
])

# Bitcoin explorer link: 
# https://blockstream.info/block/00000000000000000001733c62adb19f1b77fa0735d0e11f25af36fc9ca908a5
# 
# Kaspad mainnet launch version release: 
# https://github.com/kaspanet/kaspad/releases/tag/v0.11.0 
assert(bytes.fromhex('00000000000000000001733c62adb19f1b77fa0735d0e11f25af36fc9ca908a5') == 
       bytes([
           0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
           0x00, 0x01, 0x73, 0x3c, 0x62, 0xad, 0xb1, 0x9f,
           0x1b, 0x77, 0xfa, 0x07, 0x35, 0xd0, 0xe1, 0x1f,
           0x25, 0xaf, 0x36, 0xfc, 0x9c, 0xa9, 0x08, 0xa5,]))

# Load the full genesis header from the pre-halt store
original_genesis_header = pre_checkpoint_store.get_raw_header(original_genesis)
assert(header_hash(original_genesis_header) == original_genesis)

# Build genesis's coinbase tx. 
# Golang ref: 
# https://github.com/kaspanet/kaspad/blob/v0.11.0/domain/dagconfig/genesis.go#L47

tx = type('Transaction', (object,), {})()
tx.version = 0
tx.subnetworkID = type('SubnetworkId', (object,), {})()
tx.subnetworkID.subnetworkId = bytes.fromhex(
    '0100000000000000000000000000000000000000')
tx.inputs = []
tx.outputs = []
tx.lockTime = 0
tx.gas = 0
tx.payload = original_genesis_tx_payload

# This shows that indeed the original genesis references the
# bitcoin block mined a few minutes before launch via the coinbase tx payload
assert(
    transaction_hash(tx) == original_genesis_header.hashMerkleRoot.hash)

In [14]:
assert_cryptographic_hash_chain_to_genesis(
    pre_checkpoint_store, 
    checkpoint_hash, 
    original_genesis)

Reached the queried genesis block: 
 caeb97960a160c211a6b2196bd78399fd4c4cc5b509f55c12c8a7d815f7536ea via 5 pruning points


Finally, we create a fresh __[MuHash](http://www.robos.org/sections/software/muhash/)__ containing an empty set, and verify that the UTXO commitment in the original genesis block matches this hash. This completes check 7. and the verification process. 

In [15]:
# Show that original genesis has an empty UTXO-set commitment

# Golang ref: https://github.com/kaspanet/go-muhash/blob/main/muhash.go#L32
empty_muhash_hash = bytes([
    0x54, 0x4e, 0xb3, 0x14, 0x2c, 0x0, 0xf, 0xa, 
    0xd2, 0xc7, 0x6a, 0xc4, 0x1f, 0x42, 0x22, 0xab, 
    0xba, 0xba, 0xbe, 0xd8, 0x30, 0xee, 0xaf, 0xee, 
    0x4b, 0x6d, 0xc5, 0x6b, 0x52, 0xd5, 0xca, 0xc0])

assert(original_genesis_header.utxoCommitment.hash == empty_muhash_hash)

print(original_genesis_header.utxoCommitment.hash.hex()) 
print(empty_muhash_hash.hex())

544eb3142c000f0ad2c76ac41f4222abbababed830eeafee4b6dc56b52d5cac0
544eb3142c000f0ad2c76ac41f4222abbababed830eeafee4b6dc56b52d5cac0


Thank you for taking the time to authenticate the integrity of Kaspa!